# Human Reference Genome (hg38) Dataset Preparation

## Environment Setup

In [1]:
!pip install pyfaidx biopython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.0 MB/s eta 0:00:00


## Human Promoter Annotations from EPDnew
[source](http://reftss.clst.riken.jp/datafiles/current/human/)

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/bioproj01"
DATA_DIR = os.path.join(PROJECT_DIR, "data")
print(f"Data directory found: {PROJECT_DIR}\nContents: {os.listdir(PROJECT_DIR)}")

Mounted at /content/drive
Data directory found: /content/drive/MyDrive/bioproj01
Contents: ['data', 'results']


In [3]:
!apt-get install tree
!tree /content/drive/MyDrive/bioproj01

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (88.0 kB/s)
Selecting previously unselected package tree.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
/content/drive/MyDrive/bioproj01
├── data
│   └── hg38
│       ├── epd_collapsed_refTSS.csv
│       ├── epd_filtered_refTSS.csv
│       ├── epd_final_10k_refTSS.csv
│       ├── hg38.fa
│       ├── hg38.fa.fai
│       ├── hg38.fa.gz
│       ├── human_promoter_vs_nonpromoter_10k_400bp.csv


In [3]:
!head -n 5 /content/drive/MyDrive/bioproj01/data/hg38/refTSS_v4.1_human_coordinate.hg38.bed.txt

chromosome	start	end	refTSS_ID	score	strand
chr1	16013	16020	rfhg_1.1	1	-
chr1	29346	29366	rfhg_2.1	1	-
chr1	36521	36538	rfhg_3.1	1	+
chr1	186535	186542	rfhg_4.1	1	-


### Parse CSV Data

In [5]:
import pandas as pd

In [6]:
refTSS_path = os.path.join(DATA_DIR, "hg38/refTSS_v4.1_human_coordinate.hg38.bed.txt")

epd = pd.read_csv(
    refTSS_path,
    sep="\t",
    header=0,   # important: header exists
    usecols=["chromosome", "start", "end", "strand"],
    dtype={
        "chromosome": str,
        "start": int,
        "end": int,
        "strand": str
    }
)

epd.rename(columns={"chromosome": "chrom"}, inplace=True)
epd.head()

,chrom,start,end,strand
0,chr1,16013,16020,-
1,chr1,29346,29366,-
2,chr1,36521,36538,+
3,chr1,186535,186542,-
4,chr1,190816,190980,+


In [7]:
print("Number of refTSS promoters:", len(epd))
print(epd["strand"].value_counts())
print(epd["chrom"].unique()[:5])

Number of refTSS promoters: 241049
strand
+    126317
-    114732
Name: count, dtype: int64
['chr1' 'chr10' 'chr11' 'chr12' 'chr13']


### Filter to autosomes + chrX

In [8]:
valid_chroms = [f"chr{i}" for i in range(1, 23)] + ["chrX"]
epd_filtered = epd[epd["chrom"].isin(valid_chroms)].reset_index(drop=True)
print("After chromosome filter:", len(epd_filtered))

After chromosome filter: 239118


### Compute TSS Position

In [9]:
epd_filtered["tss"] = ((epd_filtered["start"] + epd_filtered["end"]) // 2).astype(int)
epd_filtered = epd_filtered.sort_values(
    ["chrom", "strand", "tss"]
).reset_index(drop=True)
epd_filtered.groupby(["chrom", "strand"])["tss"].is_monotonic_increasing.head()

chrom  strand
chr1   +         True
       -         True
chr10  +         True
       -         True
chr11  +         True
Name: tss, dtype: bool

### Collapse Nearby TSS

In [11]:
import numpy as np

In [12]:
def collapse_tss_clusters(df, distance=50):
    """
    Collapse refTSS entries within `distance` bp separately for each chromosome and strand.
    """
    collapsed = []
    for (chrom, strand), group in df.groupby(["chrom", "strand"]):
        current_cluster = [group.iloc[0]]
        for i in range(1, len(group)):
            prev = current_cluster[-1]
            curr = group.iloc[i]
            if curr["tss"] - prev["tss"] <= distance:
                current_cluster.append(curr)
            else:
                cluster_tss = int(
                    np.mean([x["tss"] for x in current_cluster])
                )
                collapsed.append({
                    "chrom": chrom,
                    "tss": cluster_tss,
                    "strand": strand,
                    "cluster_size": len(current_cluster)
                })
                current_cluster = [curr]
        cluster_tss = int(
            np.mean([x["tss"] for x in current_cluster])
        )
        collapsed.append({
            "chrom": chrom,
            "tss": cluster_tss,
            "strand": strand,
            "cluster_size": len(current_cluster)
        })
    return pd.DataFrame(collapsed)

epd_collapsed = collapse_tss_clusters(epd_filtered, distance=50)

In [13]:
print("After collapsing:", len(epd_collapsed))
print(epd_collapsed["cluster_size"].describe())

After collapsing: 186356
count    186356.000000
mean          1.283125
std           1.449386
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         380.000000
Name: cluster_size, dtype: float64


### Subsample to Colab-safe size

In [14]:
epd_final = epd_collapsed.sample(n=10000, random_state=42).reset_index(drop=True)

print("Final promoter count:", len(epd_final))
epd_final.head()

Final promoter count: 10000


,chrom,tss,strand,cluster_size
0,chr3,188060753,-,1
1,chr17,44219923,-,1
2,chr1,206684988,+,1
3,chr4,26288442,-,1
4,chr6,28511572,+,1


### Save Processed Annotation Files

In [15]:
epd_filtered.to_csv(f"{DATA_DIR}/hg38/epd_filtered_refTSS.csv", index=False)
epd_collapsed.to_csv(f"{DATA_DIR}/hg38/epd_collapsed_refTSS.csv", index=False)
epd_final.to_csv(f"{DATA_DIR}/hg38/epd_final_10k_refTSS.csv", index=False)

## Load the Human Reference Gnome HG38
[source](https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz)

In [16]:
from pyfaidx import Fasta

In [17]:
HG38_PATH = os.path.join(DATA_DIR, "hg38/hg38.fa")

genome = Fasta(HG38_PATH)

print("Chromosomes loaded:", list(genome.keys())[:5])

Chromosomes loaded: ['chr1', 'chr10', 'chr11', 'chr11_KI270721v1_random', 'chr12']


### Extract Promoter Sequences (±200 bp)

In [19]:
from tqdm import tqdm

In [20]:
PROMOTER_HALF_WINDOW = 200

def extract_promoter(chrom, tss, strand, genome, upstream, downstream):
    try:
        if strand == "+":
            start = tss - upstream
            end = tss + downstream
            seq = genome[chrom][start:end].seq
        else:
            start = tss - downstream
            end = tss + upstream
            seq = genome[chrom][start:end].reverse.complement.seq

        if len(seq) != upstream + downstream:
            return None
        if "N" in seq.upper():
            return None
        return seq.upper()
    except Exception:
        return None

sequences = []
valid_rows = []

for _, row in tqdm(epd_final.iterrows(), total=len(epd_final)):
    seq = extract_promoter(
        row["chrom"],
        row["tss"],
        row["strand"],
        genome,
        PROMOTER_HALF_WINDOW,
        PROMOTER_HALF_WINDOW
    )
    if seq is not None:
        sequences.append(seq)
        valid_rows.append(row)

len(sequences)

100%|██████████| 10000/10000 [01:25<00:00, 116.88it/s]


9997

### Build Promoter Dataset

In [22]:
promoter_df = pd.DataFrame(valid_rows).reset_index(drop=True)
promoter_df["sequence"] = sequences

print("Final usable promoters:", len(promoter_df))
promoter_df.head()

print(promoter_df["sequence"].str.len().value_counts())
print(promoter_df["strand"].value_counts())

Final usable promoters: 9997
sequence
400    9997
Name: count, dtype: int64
strand
+    5228
-    4769
Name: count, dtype: int64


## Generate Negative Samples (Non-Promoters)

### Build Promoter Interval Mask

In [23]:
def promoter_interval_mask(df):
  promoter_intervals = {}
  for _, row in df.iterrows():
      chrom = row["chrom"]
      start = row["tss"] - PROMOTER_HALF_WINDOW
      end = row["tss"] + PROMOTER_HALF_WINDOW
      promoter_intervals.setdefault(chrom, []).append((start, end))
  for chrom in promoter_intervals:
      promoter_intervals[chrom].sort()
  return promoter_intervals

def overlaps_any(start, end, intervals):
    for s, e in intervals:
        if start < e and end > s:
            return True
    return False

def gc_content(seq):
    seq = seq.upper()
    return (seq.count("G") + seq.count("C")) / len(seq)

promoter_gc = promoter_df["sequence"].apply(gc_content)

print(promoter_gc.describe())

count    9997.000000
mean        0.522735
std         0.128345
min         0.197500
25%         0.422500
50%         0.507500
75%         0.617500
max         0.900000
Name: sequence, dtype: float64


### Sample Negative Sequences

In [25]:
import random

In [26]:
NEGATIVE_TARGET = len(promoter_df)
SEQ_LEN = 2 * PROMOTER_HALF_WINDOW

promoter_intervals = promoter_interval_mask(promoter_df)
chromosomes = promoter_df["chrom"].unique()

negative_sequences = []
negative_gc = []

while len(negative_sequences) < NEGATIVE_TARGET:
    chrom = random.choice(chromosomes)
    chrom_len = len(genome[chrom])

    start = random.randint(0, chrom_len - SEQ_LEN)
    end = start + SEQ_LEN

    if overlaps_any(start, end, promoter_intervals.get(chrom, [])):
        continue

    seq = genome[chrom][start:end].seq.upper()

    if "N" in seq:
        continue

    gc = gc_content(seq)

    # loose GC matching (±10%)
    if abs(gc - promoter_gc.mean()) > 0.10:
        continue

    negative_sequences.append(seq)
    negative_gc.append(gc)

len(negative_sequences)

9997

### Build Negative DataFrame

In [28]:
negative_df = pd.DataFrame({
    "chrom": ["NA"] * len(negative_sequences),
    "tss": [-1] * len(negative_sequences),
    "strand": ["+"] * len(negative_sequences),
    "cluster_size": [0] * len(negative_sequences),
    "sequence": negative_sequences
})

print("Negative samples:", len(negative_df))
print(negative_df["sequence"].str.len().value_counts())

print("Promoter GC:", promoter_gc.mean())
print("Negative GC:", pd.Series(negative_gc).mean())

Negative samples: 9997
sequence
400    9997
Name: count, dtype: int64
Promoter GC: 0.5227348204461338
Negative GC: 0.4877030609182754


## Final Labeled Dataset

In [29]:
promoter_df["label"] = 1
negative_df["label"] = 0

final_df = pd.concat(
    [promoter_df, negative_df],
    ignore_index=True
).sample(frac=1, random_state=42).reset_index(drop=True)

print(final_df["label"].value_counts())

label
0    9997
1    9997
Name: count, dtype: int64


In [30]:
final_df.to_csv(
    f"{DATA_DIR}/hg38/human_promoter_vs_nonpromoter_10k_400bp.csv",
    index=False
)